In [1]:
import tensorflow as tf
import config
import numpy as np
import os
import model
import glob


In [2]:
def main():
    print("Running...", config.config)
    # Set Memory Growth
    if config.config["allow_growth"]:
        config_proto = tf.compat.v1.ConfigProto()
        config_proto.gpu_options.allow_growth = True
        session = tf.compat.v1.Session(config=config_proto)
        tf.compat.v1.keras.backend.set_session(session)

    # Set GPU visibility
    gpus = config.config["train"]["gpu"]
    if gpus == [0, 1]:
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpus[0])

    epochs = config.config["train"]["epochs"]
    unfreeze_epoch = config.config['train']['unfreeze_epoch']
    pretrained = config.config['train']['pretrained']



    _model = model.Module()
    initial_epoch = 0
    weight = config.config['train']['pretrained_weight']
    if pretrained:
        _model.load_pretrained(weight)
        initial_epoch = config.config['train']['pretrained_epoch']

    
    import pandas as pd
    data = pd.read_excel(r"F:\Lab\datasets\visob\demographic information_VISOB.xlsx")

In [11]:
import pandas as pd
data = pd.read_excel(r"F:\Lab\datasets\visob\demographic information_VISOB.xlsx")
data.head()
data.ID

0      1141
1      1142
2      1144
3      1147
4      1148
       ... 
578    6000
579    6001
580    6002
581    6003
582    6004
Name: ID, Length: 583, dtype: int64

In [ ]:
def setup_config():
    model = config.config["model"]
    class_type = config.config["class"]
    finetune = "finetuned" if config.config["fine-tune"] else "base"
    input_type = config.config["input"]["type"]
    augmented = "aug" if config.config["input"]["randaugment"] else "noaug"
    ratio = config.config["ratio"]

    run_id = '_'.join([model, "VISOB", input_type,
                      augmented, class_type, finetune, ratio])
                      
    # run_id = '_'.join([model, "UFPR", input_type,
    #                   augmented, class_type, finetune])
    config.config["run_id"] = run_id


models = ["EfficientNetB4", "DenseNet121", "MobileNetV2", "InceptionV3",
            "InceptionResNetV2", "ResNet50", "VGG19", "Xception", "EfficientNetB0"]

ckpt_dir = config.config["train"]["ckpt_dir"]

config.config["input"]["randaugment"] = True
    
config.config["mode"] = "train"
class_type = "gender"
for class_type in ["gender"]:
    config.config["class"] = class_type

    # run base models
    config.config["fine-tune"] = False
    for model in models:
        config.config["model"] = model
        setup_config()
        main.main()

    # Run fine tuned
    config.config["fine-tune"] = True
    for model in models:
        config.config["model"] = model
        setup_config()
        # set pretrained weights
        files = glob.glob(os.path.join(
            ckpt_dir, config.config["run_id"].replace("finetuned", "base") + "*"))
        files.sort()
        config.config["train"]["pretrained_weight"] = files[-1]
        main()
